# Imports and initialization of general parameters
***

In [9]:
from helpers.pareto_fairness import compute_pareto_metrics
from config.info import AGES, RACES, GENDERS, COMBS_BASELINE, COMBS_FOULDS, COMBS_MARTINEZ
from visualization.subgroup_distribution import plot_dist
from dataprocess.dataloader import load_data
from dataprocess.dataclass import Data
from config.get_args import get_args
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from lightning import seed_everything
from plotly.subplots import make_subplots
from helpers.init import init
import numpy as np
import argparse
import pandas as pd
import random
import torch.nn as nn
import torch
import torchmetrics as tm
import pickle

loss_fct = nn.CrossEntropyLoss()
protected_atts = ['age_', 'race_', 'gender_']


# Auto reload part
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
def plot_results(df_b : pd.DataFrame, df_f : pd.DataFrame, df_d : pd.DataFrame, df_m : pd.DataFrame, filename : str):
    # Initialization
    df_all = pd.concat([df_b, df_f, df_d, df_m])
    task_cc = 'cancer_classification'
    fig = make_subplots(rows = 2, cols = 1, vertical_spacing = 0.2,
                        subplot_titles = ('Accuracy', 'MMPF_size'))
    
    # Traces
    acc = px.bar(df_all[df_all.task == task_cc], x = 'cancer', y = ['Accuracy'], color = 'model',
                color_discrete_sequence = ['rgb(170, 220, 225)', 'rgb(80, 135, 180)', 'rgb(50, 100, 170)', 'rgb(25, 35, 90)'], text_auto = '.3f')
    # score_0 = px.bar(df_all[df_all.task == task_cc], x = 'cancer', y = ['F1-score_0'], color = 'model',
    #             color_discrete_sequence = ['rgb(25, 35, 90)', 'rgb(80, 135, 180)'], text_auto = '.3f').update_traces(showlegend = False)
    # score_1 = px.bar(df_all[df_all.task == task_cc], x = 'cancer', y = ['F1-score_1'], color = 'model',
    #             color_discrete_sequence = ['rgb(25, 35, 90)', 'rgb(80, 135, 180)'], text_auto = '.3f').update_traces(showlegend = False)
    mmpf = px.bar(df_all[df_all.task == task_cc], x = 'cancer', y = ['MMPF_size'], color = 'model',
                color_discrete_sequence = ['rgb(170, 220, 225)', 'rgb(80, 135, 180)', 'rgb(50, 100, 170)', 'rgb(25, 35, 90)'], text_auto = '.3f').update_traces(showlegend = False)

    # Update the figure
    for t in range(len(acc['data'])): fig.add_trace(acc['data'][t], row = 1, col = 1)
    # for t in range(len(score_0['data'])): fig.add_trace(score_0['data'][t], row = 2, col = 1)
    # for t in range(len(score_1['data'])): fig.add_trace(score_1['data'][t], row = 3, col = 1)
    for t in range(len(mmpf['data'])): fig.add_trace(mmpf['data'][t], row = 2, col = 1)
    fig.update_layout(height = 600, width = 1200, 
                    template = 'none',
                    # xaxis = {'title': 'Cancer'},
                    yaxis = {'title': 'Values [no unit]'},
                    # xaxis2 = {'title': 'Cancer'},
                    yaxis2 = {'title': 'Values [no unit]'},
                    # xaxis3 = {'title': 'Cancer'},
                    yaxis3 = {'title': 'Values [no unit]'},
                    # xaxis4 = {'title': 'Cancer'},
                    yaxis4 = {'title': 'Values [no unit]'})
    fig.update_layout(barmode = 'group', bargap = 0.3, bargroupgap = 0.1)
    fig.write_image(filename)
    fig.show()

# Baseline
***

In [11]:
def keep_best(df100 : pd.DataFrame, df1000 : pd.DataFrame):
    # Check the match between the cancer
    cs = [c for c in df100.cancer.to_list() if c in df1000.cancer.to_list()]
    df100.set_index('cancer', drop = False, inplace = True)
    df1000.set_index('cancer', drop = False, inplace = True)
    df = df100[df100.cancer.isin(cs)].copy()
    df['ID'] = 100
    
    # Check the best MMPF_size value between df100 and df1000, and add it to a new df
    for c in cs:
        
        # Compare the values 
        v1 = df100[df100.cancer == c]['MMPF_size'][0]
        v2 = df1000[df1000.cancer == c]['MMPF_size'][0]
        if v2 <= v1:
            df.loc[c, 'Accuracy'] = df1000[df1000.cancer == c]['Accuracy'][0]
            df.loc[c, 'F1-score_0'] = df1000[df1000.cancer == c]['F1-score_0'][0]
            df.loc[c, 'F1-score_1'] = df1000[df1000.cancer == c]['F1-score_1'][0]
            df.loc[c, 'MMPF_size'] = df1000[df1000.cancer == c]['MMPF_size'][0]
            df.loc[c, 'ID'] = 1000
    return df

def select_best_test(df_val, df100, df1000):
    # Copy the df
    cs = [c for c in df100.cancer.to_list() if c in df1000.cancer.to_list()]
    df100.set_index('cancer', drop = False, inplace = True)
    df1000.set_index('cancer', drop = False, inplace = True)
    df = df_val.copy()

    # Select the best inside the test df depending of the df val
    for c in cs:
        
        # Check which ID is the best validation MMPF value
        if df.loc[c, 'ID'] == 100:
            df.loc[c, 'Accuracy'] = df100[df100.cancer == c]['Accuracy'][0]
            df.loc[c, 'F1-score_0'] = df100[df100.cancer == c]['F1-score_0'][0]
            df.loc[c, 'F1-score_1'] = df100[df100.cancer == c]['F1-score_1'][0]
            df.loc[c, 'MMPF_size'] = df100[df100.cancer == c]['MMPF_size'][0]
        else:
            df.loc[c, 'Accuracy'] = df1000[df1000.cancer == c]['Accuracy'][0]
            df.loc[c, 'F1-score_0'] = df1000[df1000.cancer == c]['F1-score_0'][0]
            df.loc[c, 'F1-score_1'] = df1000[df1000.cancer == c]['F1-score_1'][0]
            df.loc[c, 'MMPF_size'] = df1000[df1000.cancer == c]['MMPF_size'][0]
            
    return df

In [50]:
def check_results_baseline(task : str, 
                           cancer : str, 
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_0/Baseline'
    results = pd.read_pickle(preds_path + f'/{task}/{cancer}/best_results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df

# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_baseline = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_BASELINE[2:]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    
    # Get the metrics - test
    df_baseline = check_results_baseline(task, cancer, df_baseline, 'test_', 1000)
df_baseline['model'] = 'Baseline'

# Plot
# plot_results_baseline(df_baseline, 'images/results_baseline.eps')

['cancer_classification', 'kich_kirc_FS']
['cancer_classification', 'kich_kirc_PM']
['cancer_classification', 'kich_kirp_FS']
['cancer_classification', 'kich_kirp_PM']
['cancer_classification', 'kirc_kirp_FS']
['cancer_classification', 'kirc_kirp_PM']
['cancer_classification', 'luad_lusc_FS']
['cancer_classification', 'luad_lusc_PM']
['tumor_detection', 'brca']
['tumor_detection', 'coad']
['tumor_detection', 'kich']
['tumor_detection', 'kirc']
['tumor_detection', 'kirp']
['tumor_detection', 'luad']
['tumor_detection', 'lusc']
['tumor_detection', 'read']


In [51]:
df_baseline

,task,cancer,Accuracy,F1-score_0,F1-score_1,MMPF_size,model
0,cancer_classification,kich_kirc_FS,0.971831,0.888889,0.983871,0.467143,Baseline
0,cancer_classification,kich_kirc_PM,0.961661,0.899160,0.976331,0.353895,Baseline
0,cancer_classification,kich_kirp_FS,0.977956,0.962457,0.984397,0.676320,Baseline
0,cancer_classification,kich_kirp_PM,0.992288,0.986175,0.994652,0.328728,Baseline
0,cancer_classification,kirc_kirp_FS,0.987324,0.991509,0.975000,0.364493,Baseline
0,cancer_classification,kirc_kirp_PM,0.988764,0.991254,0.984293,0.387133,Baseline
0,cancer_classification,luad_lusc_FS,0.929032,0.933434,0.924007,0.462914,Baseline
0,cancer_classification,luad_lusc_PM,0.953984,0.958877,0.947771,0.493846,Baseline
0,tumor_detection,brca,0.996538,0.992021,0.997789,0.344031,Baseline
0,tumor_detection,coad,0.998106,0.990291,0.998951,0.396591,Baseline


# EDF
***

In [13]:
def check_results_foulds(task : str, 
                           cancer : str, 
                           lambda_ : float,
                           pt_method : str,
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer], 'lambda_' : [lambda_], 'pt_method' : [pt_method]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_0/Foulds'
    results = pd.read_pickle(preds_path + f'/{task}/{cancer}/lambda_{lambda_}/{pt_method}/best_results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df


def select_hyper(df_foulds):
    # Initialization
    df_foulds.reset_index(inplace = True, drop = True)
    cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
    df_foulds_best = pd.DataFrame(columns = cols)

    # Loop on the combinations for baseline
    for comb in COMBS_BASELINE[2:10]:
        
        # Extract combinations
        task = comb[0]
        cancer = comb[1]
        cond = (df_foulds.task == task) & (df_foulds.cancer == cancer)
        
        # Get the metrics from the models with the best hyperparameters
        # Get hyperparameters
        best_idx = df_foulds[cond].MMPF_size.argmin()
        sub_dict = {'task' : [task], 'cancer' : [cancer]}
        sub_dict['Accuracy'] = df_foulds[cond].iloc[best_idx]['Accuracy']
        sub_dict['F1-score_0'] = df_foulds[cond].iloc[best_idx]['F1-score_0']
        sub_dict['F1-score_1'] = df_foulds[cond].iloc[best_idx]['F1-score_1']
        sub_dict['MMPF_size'] = df_foulds[cond].iloc[best_idx]['MMPF_size']
        
        # Update the dataframe
        if df_foulds_best.empty: df_foulds_best = pd.DataFrame(data = sub_dict)
        else: df_foulds_best = pd.concat([df_foulds_best, pd.DataFrame(data = sub_dict)])
    
    return df_foulds_best


# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_foulds_100 = pd.DataFrame(columns = cols)
df_foulds_1000 = pd.DataFrame(columns = cols)
df_foulds_100_test = pd.DataFrame(columns = cols)
df_foulds_1000_test = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_FOULDS[2:10]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    l = comb[2]
    pt_method = comb[3]
    
    # Get the metrics
    df_foulds_100 = check_results_foulds(task, cancer, l, pt_method, df_foulds_100, 'val_', 100)
    df_foulds_1000 = check_results_foulds(task, cancer, l, pt_method, df_foulds_1000, 'val_', 1000)
    
    # Get the metrics
    df_foulds_100_test = check_results_foulds(task, cancer, l, pt_method, df_foulds_100_test, 'test_', 100)
    df_foulds_1000_test = check_results_foulds(task, cancer, l, pt_method, df_foulds_1000_test, 'test_', 1000)
    
# # Select hyperparameters
# df_foulds_100_ = select_hyper(df_foulds_100)
# df_foulds_1000_ = select_hyper(df_foulds_1000)
# df_foulds_100_test_ = select_hyper(df_foulds_100_test)
# df_foulds_1000_test_ = select_hyper(df_foulds_1000_test)

df_f = keep_best(df_foulds_100, df_foulds_1000)
df_f_test = select_best_test(df_f, df_foulds_100_test, df_foulds_1000_test)
df_f_test['model'] = 'EDF'
# plot_results_others(df_baseline_1000_test, df_f_test, 'EDF', 'images/results_foulds.eps')

['cancer_classification', 'kich_kirc_FS', 0.01, 'DF_pos']
['cancer_classification', 'kich_kirc_PM', 0.01, 'DF_pos']
['cancer_classification', 'kich_kirp_FS', 0.01, 'DF_pos']
['cancer_classification', 'kich_kirp_PM', 0.01, 'DF_pos']
['cancer_classification', 'kirc_kirp_FS', 0.01, 'DF_pos']
['cancer_classification', 'kirc_kirp_PM', 0.01, 'DF_pos']
['cancer_classification', 'luad_lusc_FS', 0.01, 'DF_pos']
['cancer_classification', 'luad_lusc_PM', 0.01, 'DF_pos']


/tmp/ipykernel_16672/2845827234.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v1 = df100[df100.cancer == c]['MMPF_size'][0]
/tmp/ipykernel_16672/2845827234.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v2 = df1000[df1000.cancer == c]['MMPF_size'][0]
/tmp/ipykernel_16672/2845827234.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[c, 'Accuracy'] = df1000[df1000.cancer == c]['Accuracy'][0]
/tmp/ipykernel_16672/

In [14]:
df_f_test

,task,cancer,lambda_,pt_method,Accuracy,F1-score_0,F1-score_1,MMPF_size,ID,model
cancer,,,,,,,,,,
kich_kirc_FS,cancer_classification,kich_kirc_FS,0.01,DF_pos,0.978459,0.910959,0.987747,0.467388,100,EDF
kich_kirc_PM,cancer_classification,kich_kirc_PM,0.01,DF_pos,0.968051,0.913793,0.980392,0.364688,100,EDF
kich_kirp_FS,cancer_classification,kich_kirp_FS,0.01,DF_pos,0.981964,0.969072,0.987270,0.676838,100,EDF
kich_kirp_PM,cancer_classification,kich_kirp_PM,0.01,DF_pos,0.997429,0.995349,0.998224,0.314361,1000,EDF
kirc_kirp_FS,cancer_classification,kirc_kirp_FS,0.01,DF_pos,0.988028,0.991992,0.976290,0.313430,100,EDF
kirc_kirp_PM,cancer_classification,kirc_kirp_PM,0.01,DF_pos,0.987516,0.990329,0.982394,0.319233,1000,EDF
luad_lusc_FS,cancer_classification,luad_lusc_FS,0.01,DF_pos,0.929839,0.934240,0.924806,0.446301,1000,EDF
luad_lusc_PM,cancer_classification,luad_lusc_PM,0.01,DF_pos,0.950617,0.955823,0.944020,0.491511,1000,EDF


# MinimaxFair
***

In [15]:
def check_results_diana(task : str, 
                           cancer : str, 
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_0/Diana'
    results = pd.read_pickle(preds_path + f'/{task}/{cancer}/results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df

# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_diana_100 = pd.DataFrame(columns = cols)
df_diana_1000 = pd.DataFrame(columns = cols)
df_diana_100_test = pd.DataFrame(columns = cols)
df_diana_1000_test = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_BASELINE[2:10]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    
    # Get the metrics
    df_diana_100 = check_results_diana(task, cancer, df_diana_100, 'val_', 100)
    df_diana_1000 = check_results_diana(task, cancer, df_diana_1000, 'val_', 1000)
    
    # Get the metrics
    df_diana_100_test = check_results_diana(task, cancer, df_diana_100_test, 'test_', 100)
    df_diana_1000_test = check_results_diana(task, cancer, df_diana_1000_test, 'test_', 1000)

df_d = keep_best(df_diana_100, df_diana_1000)
df_d_test = select_best_test(df_d, df_diana_100_test, df_diana_1000_test)
df_d_test['model'] = 'MinimaxFair'
# plot_results_others(df_baseline_1000_test, df_d_test, 'MinimaxFair', 'images/results_diana.eps')

['cancer_classification', 'kich_kirc_FS']
['cancer_classification', 'kich_kirc_PM']
['cancer_classification', 'kich_kirp_FS']
['cancer_classification', 'kich_kirp_PM']
['cancer_classification', 'kirc_kirp_FS']
['cancer_classification', 'kirc_kirp_PM']
['cancer_classification', 'luad_lusc_FS']
['cancer_classification', 'luad_lusc_PM']


/tmp/ipykernel_16672/2845827234.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v1 = df100[df100.cancer == c]['MMPF_size'][0]
/tmp/ipykernel_16672/2845827234.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v2 = df1000[df1000.cancer == c]['MMPF_size'][0]
/tmp/ipykernel_16672/2845827234.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[c, 'Accuracy'] = df1000[df1000.cancer == c]['Accuracy'][0]
/tmp/ipykernel_16672/

In [30]:
df_d_test

,task,cancer,Accuracy,F1-score_0,F1-score_1,MMPF_size,ID,model
cancer,,,,,,,,
kich_kirc_FS,cancer_classification,kich_kirc_FS,0.977630,0.909091,0.987246,0.467116,100,MinimaxFair
kich_kirc_PM,cancer_classification,kich_kirc_PM,0.961661,0.891892,0.976699,0.391527,100,MinimaxFair
kich_kirp_FS,cancer_classification,kich_kirp_FS,0.973948,0.954386,0.981767,0.581539,100,MinimaxFair
kich_kirp_PM,cancer_classification,kich_kirp_PM,0.989717,0.981481,0.992883,0.313285,100,MinimaxFair
kirc_kirp_FS,cancer_classification,kirc_kirp_FS,0.979577,0.986340,0.959554,0.366988,100,MinimaxFair
kirc_kirp_PM,cancer_classification,kirc_kirp_PM,0.968789,0.975892,0.955752,0.403128,1000,MinimaxFair
luad_lusc_FS,cancer_classification,luad_lusc_FS,0.926613,0.932342,0.919824,0.533090,100,MinimaxFair
luad_lusc_PM,cancer_classification,luad_lusc_PM,0.942761,0.948537,0.935525,0.502044,100,MinimaxFair


# APStar
***

In [17]:
def check_results_martinez(task : str, 
                           cancer : str, 
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_0/Martinez'
    # if ID == 100 : results = pd.read_pickle(preds_path + f'/{task}/{cancer}/alpha_0.5/results.pkl')
    results = pd.read_pickle(preds_path + f'/{task}/{cancer}/results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df

# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_martinez_100 = pd.DataFrame(columns = cols)
df_martinez_1000 = pd.DataFrame(columns = cols)
df_martinez_100_test = pd.DataFrame(columns = cols)
df_martinez_1000_test = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_BASELINE[2:10]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    
    # Get the metrics
    df_martinez_100 = check_results_martinez(task, cancer, df_martinez_100, 'val_', 100)
    df_martinez_1000 = check_results_martinez(task, cancer, df_martinez_1000, 'val_', 1000)

    # Get the metrics
    df_martinez_100_test = check_results_martinez(task, cancer, df_martinez_100_test, 'test_', 100)
    df_martinez_1000_test = check_results_martinez(task, cancer, df_martinez_1000_test, 'test_', 1000)
    
df_m = keep_best(df_martinez_100, df_martinez_1000)
df_m_test = select_best_test(df_m, df_martinez_100_test, df_martinez_1000_test)
df_m_test['model'] = 'APStar'
# plot_results_others(df_baseline_1000_test, df_m_test, 'APStar', 'images/results_martinez.eps')

['cancer_classification', 'kich_kirc_FS']
['cancer_classification', 'kich_kirc_PM']
['cancer_classification', 'kich_kirp_FS']
['cancer_classification', 'kich_kirp_PM']
['cancer_classification', 'kirc_kirp_FS']
['cancer_classification', 'kirc_kirp_PM']
['cancer_classification', 'luad_lusc_FS']
['cancer_classification', 'luad_lusc_PM']


/tmp/ipykernel_16672/2845827234.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v1 = df100[df100.cancer == c]['MMPF_size'][0]
/tmp/ipykernel_16672/2845827234.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v2 = df1000[df1000.cancer == c]['MMPF_size'][0]
/tmp/ipykernel_16672/2845827234.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[c, 'Accuracy'] = df1000[df1000.cancer == c]['Accuracy'][0]
/tmp/ipykernel_16672/

In [18]:
df_m_test

,task,cancer,Accuracy,F1-score_0,F1-score_1,MMPF_size,ID,model
cancer,,,,,,,,
kich_kirc_FS,cancer_classification,kich_kirc_FS,0.970174,0.880000,0.982971,0.467123,100,APStar
kich_kirc_PM,cancer_classification,kich_kirc_PM,0.960064,0.895397,0.975321,0.313264,1000,APStar
kich_kirp_FS,cancer_classification,kich_kirp_FS,0.981964,0.968641,0.987342,0.584425,100,APStar
kich_kirp_PM,cancer_classification,kich_kirp_PM,0.989717,0.981481,0.992883,0.313262,100,APStar
kirc_kirp_FS,cancer_classification,kirc_kirp_FS,0.971127,0.980724,0.942496,0.352675,1000,APStar
kirc_kirp_PM,cancer_classification,kirc_kirp_PM,0.970037,0.976517,0.958621,0.467098,1000,APStar
luad_lusc_FS,cancer_classification,luad_lusc_FS,0.874194,0.886463,0.858951,0.591313,100,APStar
luad_lusc_PM,cancer_classification,luad_lusc_PM,0.891134,0.899273,0.881563,0.496515,1000,APStar


# Plot all results 
***

In [47]:
plot_results(df_baseline, df_f_test, df_d_test, df_m_test, 'images/atts_0.eps')

# In-depth analysis of kich_kirc_PM
***

In [26]:
def compute_density_and_metrics(results, results_all):
    # Initialization
    results = results.astype({'age_' : 'int32', 
                            'race_' : 'int32', 
                            'gender_' : 'int32', 
                            'pred' : 'int32', 
                            'label' : 'int32'})
    atts = ['age_', 'race_', 'gender_']
    subgroups = np.unique(results[atts].values, axis = 0)
    df = pd.DataFrame(subgroups, columns = atts)
    df['Accuracy'] = 0.0
    df['F1-score_0'] = 0.0
    df['F1-score_1'] = 0.0
    df['density'] = 0

    # Loop on all the subgroups
    for idx, sg in enumerate(subgroups):
        cond = (results.age_ == sg[0]) & (results.race_ == sg[1]) & (results.gender_ == sg[2])
        sub_df = results[cond]
        
        # Accuracy
        df.loc[idx, 'Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(sub_df.pred.values), torch.Tensor(sub_df.label.values)).item()
        
        # Compute the f1-score
        f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(sub_df.pred.values), torch.Tensor(sub_df.label.values))
        df.loc[idx, 'F1-score_0'] = f1_scores[0].item()
        df.loc[idx, 'F1-score_1'] = f1_scores[1].item()
        
        # Density
        df.loc[idx, 'density'] = len(sub_df)
    
    mmpf_metrics = compute_pareto_metrics(results_all, protected_atts)
        
    # Modifications of the refs 
    df['age_str'] = df.age_.astype(str)
    df['race_str'] = df.apply(lambda row : RACES['str'][row['race_']], axis = 1)
    df['gender_str'] = df.apply(lambda row : GENDERS['str'][row['gender_']], axis = 1)
    return df, mmpf_metrics

In [34]:
# Initialization
task = 'cancer_classification'
cancer = 'kich_kirp_FS'

# Get the Baseline results 
ID_b = 1000
preds_path = f'results/preds/run_{ID_b}/add_protected_atts_1/Baseline/{task}/{cancer}/best_results.pkl'
results_all_b = pd.read_pickle(preds_path)
results_b = results_all_b[results_all_b.set == 'test']

# Get the Martinez results 
ID_m = 100
preds_path = f'results/preds/run_{ID_m}/add_protected_atts_1/Martinez/{task}/{cancer}/results.pkl'
results_all_m = pd.read_pickle(preds_path)
results_m = results_all_m[results_all_m.set == 'test']

In [35]:
# Get the metrics
df_b, b = compute_density_and_metrics(results_b, results_all_b)
df_m, m = compute_density_and_metrics(results_m, results_all_m)

# Pivot tables
d0 = pd.pivot_table(df_b[df_b.gender_ == 0][['age_str', 'race_str', 'density']], index = 'age_str', columns = 'race_str')['density']
d1 = pd.pivot_table(df_b[df_b.gender_ == 1][['age_str', 'race_str', 'density']], index = 'age_str', columns = 'race_str')['density']
# density = pd.pivot_table(df_b[df_b.race_ == 1][['age_str', 'gender_str', 'density']], index = 'age_str', columns = 'gender_str')
metric = 'Accuracy'
baseline_0 = pd.pivot_table(df_b[df_b.gender_ == 0][['age_str', 'race_str', metric]], index = 'age_str', columns = 'race_str')[metric]
martinez_0 = pd.pivot_table(df_m[df_m.gender_ == 0][['age_str', 'race_str', metric]], index = 'age_str', columns = 'race_str')[metric]
baseline_1 = pd.pivot_table(df_b[df_b.gender_ == 1][['age_str', 'race_str', metric]], index = 'age_str', columns = 'race_str')[metric]
martinez_1 = pd.pivot_table(df_m[df_m.gender_ == 1][['age_str', 'race_str', metric]], index = 'age_str', columns = 'race_str')[metric]

In [48]:
# Build the subplot
fig = make_subplots(2, 3, specs=[[{'r':0.02}, {'l':0.02}, {'l':0.02}],
                                 [{'r':0.02}, {'l':0.02}, {'l':0.02}]], #horizontal_spacing = 0.1,
                    subplot_titles=('Distribution of the data (FEMALE)', 'Accuracy for Baseline (FEMALE)', 'Accuracy for APStar (FEMALE)', 
                                    'Distribution of the data (MALE)', 'Accuracy for Baseline (MALE)', 'Accuracy for APStar (MALE)'))

# Add the heatmaps
fig.add_trace(go.Heatmap(x = d0.columns, 
                         y = d0.index, 
                         z = d0, 
                         coloraxis = 'coloraxis', texttemplate = '%{z:.2f}'), 1, 1)
fig.add_trace(go.Heatmap(x = baseline_0.columns, 
                         y = baseline_0.index, 
                         z = baseline_0, 
                         coloraxis = 'coloraxis2', texttemplate = '%{z:.2f}'), 1, 2)
fig.add_trace(go.Heatmap(x = martinez_0.columns,
                         y = martinez_0.index, 
                         z = martinez_0, 
                         coloraxis = 'coloraxis2', texttemplate = '%{z:.2f}'), 1, 3)
fig.add_trace(go.Heatmap(x = d1.columns, 
                         y = d1.index, 
                         z = d1, 
                         coloraxis = 'coloraxis', texttemplate = '%{z:.2f}'), 2, 1)
fig.add_trace(go.Heatmap(x = baseline_1.columns, 
                         y = baseline_1.index, 
                         z = baseline_1, 
                         coloraxis = 'coloraxis2', texttemplate = '%{z:.2f}'), 2, 2)
fig.add_trace(go.Heatmap(x = martinez_1.columns,
                         y = martinez_1.index, 
                         z = martinez_1, 
                         coloraxis = 'coloraxis2', texttemplate = '%{z:.2f}'), 2, 3)

# Update the figures
fig.update_layout(height = 600, width = 1500,
                  template = 'none',
                  yaxis = {"title": 'Age'},
                  yaxis2 = {"title": 'Age'},
                  yaxis3 = {"title": 'Age'},
                  yaxis4 = {"title": 'Age'},
                  yaxis5 = {"title": 'Age'},
                  yaxis6 = {"title": 'Age'},
                  coloraxis = dict(colorscale='ice', colorbar_x=0.27, colorbar_thickness=23, 
                                   colorbar_title = 'Nb of data ', colorbar_title_side = 'right'),
                  coloraxis2 = dict(colorscale='RdYlGn', colorbar_x=1, colorbar_thickness=23, 
                                    colorbar_title = 'Accuracy', colorbar_title_side = 'right'))
fig.update_layout(xaxis1 = dict(labelalias={'BLACK OR AFRICAN AMERICAN' : 'BLACK'}),
                  yaxis1 = dict(tickvals=[3, 4, 5, 6, 7, 8]),
                  xaxis2 = dict(labelalias={'BLACK OR AFRICAN AMERICAN' : 'BLACK'}),
                  yaxis2 = dict(tickvals=[3, 4, 5, 6, 7, 8]),
                  xaxis3 = dict(labelalias={'BLACK OR AFRICAN AMERICAN' : 'BLACK'}),
                  yaxis3 = dict(tickvals=[3, 4, 5, 6, 7, 8]),
                  xaxis4 = dict(labelalias={'BLACK OR AFRICAN AMERICAN' : 'BLACK'}),
                  yaxis4 = dict(tickvals=[3, 4, 5, 6, 7, 8]),
                  xaxis5 = dict(labelalias={'BLACK OR AFRICAN AMERICAN' : 'BLACK'}),
                  yaxis5 = dict(tickvals=[3, 4, 5, 6, 7, 8]),
                  xaxis6 = dict(labelalias={'BLACK OR AFRICAN AMERICAN' : 'BLACK'}),
                  yaxis6 = dict(tickvals=[3, 4, 5, 6, 7, 8]))
# fig.write_image("images/deep_analysis.eps")
fig.show()

In [44]:
fig.write_image("images/deep_analysis_atts_0.eps")

In [158]:
d0.columns.to_list()

[('density', 'ASIAN'),
 ('density', 'BLACK OR AFRICAN AMERICAN'),
 ('density', 'WHITE')]

In [160]:
pd.pivot_table(df_b[df_b.gender_ == 0][['age_str', 'race_str', 'density']], index = 'age_str', columns = 'race_str')['density']

race_str,ASIAN,BLACK OR AFRICAN AMERICAN,WHITE
age_str,,,
3,1.0,1.0,1.0
4,NaN,2.0,10.0
5,NaN,3.0,16.0
6,1.0,3.0,24.0
7,NaN,2.0,20.0
8,NaN,NaN,6.0
